In [1]:
# Import necessary libraries
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import numpy as np

# Set working directory
#os.chdir("C:/Users/Hp/Desktop/HCES2223/Data")

In [39]:
# Read the layout file which contains the structure of the fixed-width files
layout = pd.read_csv("C:/Users/Hp/Desktop/HCES2223/Resource/Layout.csv")

# Display the layout for reference
print(layout.head())

# Get the unique levels from the layout
levels = layout['Level'].unique().tolist()

   Level  Slno            Item  Length
0      1     1     Survey Name       4
1      1     2            Year       4
2      1     3  FSU Serial No.       5
3      1     4          Sector       1
4      1     5           State       2


In [25]:
# Get a list of data files
data_files = os.listdir()
data_files= data_files[0:3]+ data_files[10:11]+ data_files[13:15]
levels=levels[0:3]+ levels[10:11]+levels[13:15]

In [41]:
# Get a list of data files
data_files = os.listdir()
data_files= [data_files[10]]
levels=[levels[10]]

In [43]:
data_files

['hces22_lvl_11.TXT']

In [45]:
# Function to process each file
for i in range(len(levels)):
    current_layout = layout[layout['Level'] == levels[i]]
    # Read the fixed-width file
    columns=dict(zip(current_layout.Item, current_layout.Length))
    
    file_name = data_files[i]
    # Read the fixed-width file
    # Read the text file
    file_path = 'C://Users//Hp//Desktop//HCES2223//Data//' +  file_name  
    with open(file_path, 'r') as file:
        data = file.readlines()

    # Parse the data into a structured format
    parsed_data = []
    for line in data:
        parsed_line = []
        start = 0
        for length in columns.values():
            parsed_line.append(line[start:start + length].strip())
            start += length
        parsed_data.append(parsed_line)

    # Create a DataFrame
    df = pd.DataFrame(parsed_data, columns=[col for col in columns.keys()])
    df['HH_ID'] = df.iloc[:, :16].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

    
    
    # Save the processed data to CSV
    output_file = f"C:/Users/Hp/Desktop/HCES2223/CSV/" + file_name +'.parquet'
    df.to_parquet(output_file, index=False)
    

######Analysis###

In [3]:
Level1=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_01.TXT.parquet')
Level2=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_02.TXT.parquet')
Level3=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_03.TXT.parquet')
Level11=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_11.TXT.parquet')
Level14=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_14.TXT.parquet')
Level15=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_15.TXT.parquet')


In [5]:
Level2=Level2[Level2.columns[16:]]
Level3=Level3[Level3.columns[16:]]
Level11=Level11[Level11.columns[16:]]
Level14=Level14[Level14.columns[16:]]
Level15=Level15[Level15.columns[16:]]

In [7]:
Level11=Level11[Level11.columns[29:]]
Level11=Level11[['HH_ID','Whether household possessed one or more item as on the date of the survey- Television',
       'Whether household possessed one or more item as on the date of the survey- Radio',
       'Whether household possessed one or more item as on the date of the survey - Laptop/PC',
       'Whether household possessed one or more item as on the date of the survey- Mobile handset',
       'Whether household possessed one or more item as on the date of the survey- Bicycle',
       'Whether household possessed one or more item as on the date of the survey- Motorcycle, scooter',
       'Whether household possessed one or more item as on the date of the survey- Motor car/jeep/van',
       'Whether household possessed one or more item as on the date of the survey- Trucks',
       'Whether household possessed one or more item as on the date of the survey - Animal cart',
       'Whether household possessed one or more item as on the date of the survey- Refrigerator',
       'Whether household possessed one or more item as on the date of the survey- Washing machine',
       'Whether household possessed one or more item as on the date of the survey- Air conditioner/air cooler']]

In [9]:
Level1=Level1[['HH_ID','Sector','State','NSS-Region','District','Multiplier']]
Level3=Level3[['HH_ID','Household Type', 'Religion of the head of the household','Social Group of the head of the household', 
               'HH Size (For FDQ)','NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)']]
Level1_3=Level1.merge(Level3, on ='HH_ID', how= 'left')
Level1_311=Level1_3.merge(Level11, on ='HH_ID', how= 'left')

In [11]:
# Extract data in a single step for each household size category
HH_Visit= Level15[Level15['Questionnaire No.'].isin(['F', 'C', 'D'])]
# Pivot the household sizes into separate columns for each questionnaire category
HH_Visit = HH_Visit.pivot(index='HH_ID', columns='Questionnaire No.', values='Household size').reset_index()

In [13]:
HH_Data= Level1_311.merge(HH_Visit, on ='HH_ID', how= 'left')

In [15]:
def expense_summariser(df, item_list, multiplier, column_name_to_be_taken):
    test= df[df['Item_Code'].isin(item_list)]
    test= test.groupby('HH_ID')['Value_in_Rs'].sum() * multiplier
    test=test.reset_index(name= column_name_to_be_taken)
    df_HHID= Level14[['HH_ID']].drop_duplicates()
    test= df_HHID.merge(test, on ='HH_ID', how= 'left')
    return test

In [17]:
#Food Items
# Subset data for level 14
L14subset= Level14[['HH_ID', 'Questionnaire No.', 'Item Code', 'Value (in Rs)']]
L14subset.columns = ['HH_ID', 'Questionnaire_No', 'Item_Code', 'Value_in_Rs']
L14subset['Value_in_Rs']=L14subset['Value_in_Rs'].astype(float)
L14subset['Item_Code']=L14subset['Item_Code'].astype(int)
FoodSummary = L14subset[L14subset['Questionnaire_No'] == "F"]

# Food expenses
Food30 = expense_summariser(FoodSummary, [129, 139, 159, 179],1, "Food_30days")  # 30-day items
Food07 = expense_summariser(FoodSummary, [169, 219, 239, 249, 199, 189, 269, 279, 289, 299], 30 / 7, "Food_07days")  # 7-day items

# Total Food expenses in 30 days
Food = Food30.merge(Food07 , on ='HH_ID', how= 'left')
Food=Food.fillna(0)
Food['Food_Expense']= Food['Food_30days'] + Food['Food_07days']

C:\Users\DIID\AppData\Local\Temp\ipykernel_3700\3137296924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L14subset['Value_in_Rs']=L14subset['Value_in_Rs'].astype(float)
C:\Users\DIID\AppData\Local\Temp\ipykernel_3700\3137296924.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L14subset['Item_Code']=L14subset['Item_Code'].astype(int)


In [19]:
# Separate data into categories: Consumables
ConsumablesSummary = L14subset[L14subset['Questionnaire_No'] == "C"]
# Consumables expenses $ 539 for imputed
Consumables30 = expense_summariser(ConsumablesSummary, [349, 459, 479, 429, 519, 499, 439, 529],1,"Consumable_30days")  # 30-day items
Consumables07 = expense_summariser(ConsumablesSummary, [309, 319, 329], 30 / 7,"Consumable_07days")  # 7-day items
Consumables365 = expense_summariser(ConsumablesSummary, [409, 419, 899], 30 / 365,"Consumable_365days")  # 365-day items

Consumable = Consumables30.merge(Consumables07 , on ='HH_ID', how= 'left')
Consumable=Consumable.merge(Consumables365 , on ='HH_ID', how= 'left')
Consumable=Consumable.fillna(0)
Consumable['Consumable_Expense']= Consumable['Consumable_30days'] + Consumable['Consumable_07days'] + Consumable['Consumable_365days']

In [21]:
# Durables
DurablesSummary = L14subset[L14subset['Questionnaire_No'] == "D"]

# Durables expenses
Durables365 = expense_summariser(DurablesSummary, DurablesSummary['Item_Code'].tolist(), 30 / 365,"Durable_365days")  # 365-day items
Durables365=Durables365.fillna(0)

In [23]:
Consumption_All= Food.merge(Consumable, on ='HH_ID', how= 'left')
Consumption_All= Consumption_All.merge(Durables365, on ='HH_ID', how= 'left')

In [25]:
Consumption_All= Food.merge(Consumable, on ='HH_ID', how= 'left')
Consumption_All= Consumption_All.merge(Durables365, on ='HH_ID', how= 'left')
HH_Data=HH_Data.merge(Consumption_All[['HH_ID','Food_Expense','Consumable_Expense','Durable_365days']], on= 'HH_ID', how= 'left')
HH_Data[['C', 'D', 'F']]=HH_Data[['C', 'D', 'F']].astype(int)

In [27]:
HH_Data['Multiplier']=HH_Data['Multiplier'].astype(float)
HH_Data['Weight']=(HH_Data['Multiplier'])/100
HH_Data['TotalExpense'] = HH_Data['Food_Expense'] + (HH_Data['Consumable_Expense'])*(HH_Data['F'])/(HH_Data['C']) + (HH_Data['Durable_365days']) * (HH_Data['F']) /(HH_Data['D'])
HH_Data['MPCE'] = HH_Data['TotalExpense'] / HH_Data['F']
HH_Data['WtdTotalExpense']=HH_Data['TotalExpense']*HH_Data['Weight']
HH_Data['WtdHH_in_F']=HH_Data['F']*HH_Data['Weight']

In [29]:
(HH_Data.groupby(['Sector'])['WtdTotalExpense'].sum() / HH_Data.groupby(['Sector'])['WtdHH_in_F'].sum()).reset_index(name='MPCE')

,Sector,MPCE
0,1,3773.052260
1,2,6458.691602


In [31]:
(HH_Data.groupby(['State','Sector'])['WtdTotalExpense'].sum() / HH_Data.groupby(['State','Sector'])['WtdHH_in_F'].sum()).reset_index(name='MPCE')

,State,Sector,MPCE
0,01,1,4295.693203
1,01,2,6178.501773
2,02,1,5560.850141
3,02,2,8075.279338
4,03,1,5314.744092
...,...,...,...
67,35,2,10268.150702
68,36,1,4802.229901
69,36,2,8158.441165
70,37,1,4035.291713


In [41]:
Level2.columns

Index(['Questionnaire No.', 'Level', 'Person Srl No.',
       'Relation to head (code)', 'Gender', 'Age(in years)',
       'Marital Status (code)', 'Highest educational level attained (code)',
       'Total year of education completed',
       'Whether used internet from any location during last 30 days',
       'No. of days stayed away from home during last 30 days',
       'No. of meals usually taken in a day',
       'No. of meals taken during last 30 days from school, balwadi etc.',
       'No. of meals taken during last 30 days from employer as\r\r\nperquisites or part of wage',
       'No. of meals taken during last 30 days  others',
       'No. of meals taken during last 30 days on payment',
       'No. of meals taken during last 30 days at home',
       'Status of Member as on revisit',
       'FDQ original member(generated field)', 'Multiplier', 'HH_ID'],
      dtype='object')

In [119]:
import numpy as np

In [117]:
Level2_Subset=Level2[['HH_ID','Person Srl No.','Relation to head (code)', 'Gender', 'Age(in years)','Marital Status (code)', 'Highest educational level attained (code)',
       'Total year of education completed']]

Level2_Subset['Total year of education completed']=Level2_Subset['Total year of education completed'].replace('',np.nan).astype(float)
Level2_Subset['Total year of education completed']=Level2_Subset['Total year of education completed'].replace(np.nan,0).astype(int)
Level2_Subset['Age(in years)']=Level2_Subset['Age(in years)'].astype(int)

C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1244482126.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level2_Subset['Total year of education completed']=Level2_Subset['Total year of education completed'].replace('',np.nan).astype(float)
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1244482126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level2_Subset['Total year of education completed']=Level2_Subset['Total year of education completed'].replace(np.nan,0).astype(int)
C:\Users\Hp\AppData\Local\T

In [121]:
HHHead_Data= Level2_Subset[Level2_Subset['Relation to head (code)']=='1']

In [123]:
HHHead_Data.columns=['HH_ID', 'Person Srl No.', 'Relation to head (code)', 'Gender_HH_Head',
       'Age_HH_Head', 'Marital Status (code)',
       'Highest educational level attained (code)',
       'Total year of education completed']

In [125]:
def Ind_year_of_edu(x):
    if (x<=5):
        return '0-5 years'
    if (x >5) & (x<=9):
        return '5-9 years'
    else:
        return '>10 years'

In [127]:
def Ind_highest_level_of_eduction(x):
    if x=='01':
        return 'Not_Literate'
    if x in ['02','03']:
        return 'Literate_or_belowPrimary'
    if x in ['04','05']:
        return 'Primary_or_Middle'
    if x  in ['06','08','07','10']:
        return 'Secondary_or_higher_secondary'
    if x in ['11','12','13']:
        return 'graduate and above'

In [129]:
HHHead_Data['Ind_highest_level_of_eduction']=HHHead_Data['Highest educational level attained (code)'].apply(lambda x: Ind_highest_level_of_eduction(x))
HHHead_Data['Ind_year_of_edu']=HHHead_Data['Total year of education completed'].apply(lambda x: Ind_year_of_edu(x))

C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1153721337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHHead_Data['Ind_highest_level_of_eduction']=HHHead_Data['Highest educational level attained (code)'].apply(lambda x: Ind_highest_level_of_eduction(x))
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1153721337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHHead_Data['Ind_year_of_edu']=HHHead_Data['Total year of education completed'].apply(lambda x: Ind_year_of_edu(x))


In [131]:
#### Share of different age group in household

In [133]:
# Define age group bins and labels
bins = [0, 14, 24, 59, float('inf')]
labels = ['0-14', '15-24', '25-59', '60 and older']
# Create a new column 'Age_Group' to categorize ages into bins
Level2_Subset['Age_Group'] = pd.cut(Level2_Subset['Age(in years)'], bins=bins, labels=labels, right=True)

# Count the number of members in each age group for each household
age_group_counts = Level2_Subset.groupby(['HH_ID', 'Age_Group']).size().unstack(fill_value=0)

# Calculate the total number of members in each household
total_members = age_group_counts.sum(axis=1)

# Calculate the share of household members in each age group
age_group_shares = age_group_counts.div(total_members, axis=0)

# Rename columns for clarity
age_group_shares.columns = ['Share of HH members in 0-14',
                            'Share of HH members in 15-24',
                            'Share of HH members in 25-59',
                            'Share of HH members in 60 and older']
age_group_shares=age_group_shares.reset_index()

C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\2164088429.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level2_Subset['Age_Group'] = pd.cut(Level2_Subset['Age(in years)'], bins=bins, labels=labels, right=True)
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\2164088429.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_counts = Level2_Subset.groupby(['HH_ID', 'Age_Group']).size().unstack(fill_value=0)


In [135]:
#Joining dataset created So far

In [137]:
HH_Data = HH_Data.merge(HHHead_Data, on ='HH_ID', how= 'left')
HH_Data=HH_Data.merge(age_group_shares, on= 'HH_ID', how='left')

In [139]:
HH_Data.columns

Index(['HH_ID', 'Sector', 'State', 'NSS-Region', 'District', 'Multiplier',
       'Household Type', 'Religion of the head of the household',
       'Social Group of the head of the household', 'HH Size (For FDQ)',
       'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Whether household possessed one or more item as on the date of the survey- Television',
       'Whether household possessed one or more item as on the date of the survey- Radio',
       'Whether household possessed one or more item as on the date of the survey - Laptop/PC',
       'Whether household possessed one or more item as on the date of the survey- Mobile handset',
       'Whether household possessed one or more item as on the date of the survey- Bicycle',
       'Whether household possessed one or more item as on the date of the survey- Motorcycle, scooter',
       'Whether household possessed one or more item as on the date of the survey- Motor car/jeep/van',
       'Whether household possessed one or

In [141]:
HH_Data.columns=['HH_ID', 'Sector', 'State', 'NSS-Region', 'District', 'Multiplier',
       'Household Type', 'Religion_HH_head',
       'Social_Group_HH_head', 'HH Size (For FDQ)',
       'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
        'Is_HH_Have_Television','Is_HH_Have_Radio','Is_HH_Have_Laptop_PC','Is_HH_Have_Mobile_handset','Is_HH_Have_Bicycle',
       'Is_HH_Have_Motorcycle_scooter','Is_HH_Have_Motorcar_jeep_van','Is_HH_Have_Trucks','Is_HH_Have_Animal_cart','Is_HH_Have_Refrigerator',
       'Is_HH_Have_Washing_machine','Is_HH_Have_Airconditioner_aircooler',
        'C', 'D', 'F','Food_Expense', 'Consumable_Expense', 'Durable_365days', 'Weight',
       'TotalExpense', 'MPCE', 'WtdTotalExpense', 'WtdHH_in_F',
       'Person Srl No.', 'Relation to head (code)', 'Gender_HH_Head',
       'Age_HH_Head', 'Marital_Status_HH_head',
        'Highest educational level attained (code)',
        'Total_year_of_education_completed_HH_head','Ind_highest_level_of_eduction_HH_head',
       'Ind_year_of_edu', 'Share of HH members in 0-14',
       'Share of HH members in 15-24', 'Share of HH members in 25-59',
       'Share of HH members in 60 and older']

In [282]:
##Creating Marriage related variable
#1. Share of married members in household
#2. share of graduate and above person in household
#3. avrage year of schooling in household

In [143]:
Level2_Subset.columns

Index(['HH_ID', 'Person Srl No.', 'Relation to head (code)', 'Gender',
       'Age(in years)', 'Marital Status (code)',
       'Highest educational level attained (code)',
       'Total year of education completed', 'Age_Group'],
      dtype='object')

In [145]:
# 1. Share of currently married members in the household

Level2_Subset['Is_CurrentlyMarried'] = Level2_Subset['Marital Status (code)'].apply(lambda x: 1 if x == '2' else 0)

# Calculate the share of married members per household
CurrentlyMarried_share = Level2_Subset.groupby('HH_ID')['Is_CurrentlyMarried'].mean()

# 2. share of graduates and above in the household
Level2_Subset['Is_Graduate_Or_Above'] = Level2_Subset['Highest educational level attained (code)'].apply(lambda x: 1 if x in ['11','12','13'] else 0)

share_graduates =  Level2_Subset.groupby('HH_ID')['Is_Graduate_Or_Above'].mean()

# 3. Average years of schooling for the household
#Level2_Subset_above_15 = Level2_Subset[Level2_Subset['Age(in years)'] > 15]
average_schooling = Level2_Subset.groupby('HH_ID')['Total year of education completed'].mean()

# Combine all the results into one DataFrame
household_stats = pd.DataFrame({
    'Share of Married Members': CurrentlyMarried_share,
    'share of Graduate and Above': share_graduates,
    'average_schooling': average_schooling
})


C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\311920899.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level2_Subset['Is_CurrentlyMarried'] = Level2_Subset['Marital Status (code)'].apply(lambda x: 1 if x == '2' else 0)
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\311920899.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Level2_Subset['Is_Graduate_Or_Above'] = Level2_Subset['Highest educational level attained (code)'].apply(lambda x: 1 if x in ['11','12','13'] else 0)


In [147]:
HH_Data= HH_Data.merge(household_stats, on ='HH_ID', how= 'left')

In [149]:
HH_Data.to_excel('HH_Data_new120824.xlsx', index= False)

In [165]:
HH_Data[['Is_HH_Have_Television', 'Is_HH_Have_Radio', 'Is_HH_Have_Laptop_PC',
       'Is_HH_Have_Mobile_handset', 'Is_HH_Have_Bicycle',
       'Is_HH_Have_Motorcycle_scooter', 'Is_HH_Have_Motorcar_jeep_van',
       'Is_HH_Have_Trucks', 'Is_HH_Have_Animal_cart',
       'Is_HH_Have_Refrigerator', 'Is_HH_Have_Washing_machine',
       'Is_HH_Have_Airconditioner_aircooler']]=HH_Data[['Is_HH_Have_Television', 'Is_HH_Have_Radio', 'Is_HH_Have_Laptop_PC',
       'Is_HH_Have_Mobile_handset', 'Is_HH_Have_Bicycle',
       'Is_HH_Have_Motorcycle_scooter', 'Is_HH_Have_Motorcar_jeep_van',
       'Is_HH_Have_Trucks', 'Is_HH_Have_Animal_cart',
       'Is_HH_Have_Refrigerator', 'Is_HH_Have_Washing_machine',
       'Is_HH_Have_Airconditioner_aircooler']].replace("", "0")

In [171]:
HH_Data.columns

Index(['HH_ID', 'Sector', 'State', 'NSS-Region', 'District', 'Multiplier',
       'Household Type', 'Religion_HH_head', 'Social_Group_HH_head',
       'HH Size (For FDQ)', 'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Is_HH_Have_Television', 'Is_HH_Have_Radio', 'Is_HH_Have_Laptop_PC',
       'Is_HH_Have_Mobile_handset', 'Is_HH_Have_Bicycle',
       'Is_HH_Have_Motorcycle_scooter', 'Is_HH_Have_Motorcar_jeep_van',
       'Is_HH_Have_Trucks', 'Is_HH_Have_Animal_cart',
       'Is_HH_Have_Refrigerator', 'Is_HH_Have_Washing_machine',
       'Is_HH_Have_Airconditioner_aircooler', 'C', 'D', 'F', 'Food_Expense',
       'Consumable_Expense', 'Durable_365days', 'Weight', 'TotalExpense',
       'MPCE', 'WtdTotalExpense', 'WtdHH_in_F', 'Person Srl No.',
       'Relation to head (code)', 'Gender_HH_Head', 'Age_HH_Head',
       'Marital_Status_HH_head', 'Highest educational level attained (code)',
       'Total_year_of_education_completed_HH_head',
       'Ind_highest_level_of_eduction_

In [299]:
HH_Data_v1=HH_Data[['HH_ID', 'Sector', 'State', 'NSS-Region', 'District',
       'Household Type', 'Religion_HH_head', 'Social_Group_HH_head',
       'HH Size (For FDQ)', 'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Gender_HH_Head','Age_HH_Head', 'Marital_Status_HH_head',
    'Ind_year_of_edu','Ind_highest_level_of_eduction_HH_head', 'Share of HH members in 0-14',
       'Share of HH members in 15-24', 'Share of HH members in 25-59',
       'Share of HH members in 60 and older', 'Share of Married Members',
       'share of Graduate and Above', 'average_schooling','Weight','TotalExpense']]

In [301]:
HH_Data_v1.to_excel('HH_Data_new_v1.xlsx', index= False)

In [173]:
HH_Data_v2=HH_Data[['HH_ID', 'Sector', 'State', 'NSS-Region', 'District',
       'Household Type', 'Religion_HH_head', 'Social_Group_HH_head',
       'HH Size (For FDQ)', 'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Is_HH_Have_Television', 'Is_HH_Have_Radio', 'Is_HH_Have_Laptop_PC',
       'Is_HH_Have_Mobile_handset', 'Is_HH_Have_Bicycle',
       'Is_HH_Have_Motorcycle_scooter', 'Is_HH_Have_Motorcar_jeep_van',
       'Is_HH_Have_Trucks', 'Is_HH_Have_Animal_cart',
       'Is_HH_Have_Refrigerator', 'Is_HH_Have_Washing_machine',
       'Is_HH_Have_Airconditioner_aircooler', 'Gender_HH_Head', 'Age_HH_Head',
       'Marital_Status_HH_head',
       'Total_year_of_education_completed_HH_head',
       'Ind_highest_level_of_eduction_HH_head',
       'Share of HH members in 0-14', 'Share of HH members in 15-24',
       'Share of HH members in 25-59', 'Share of HH members in 60 and older',
       'Share of Married Members', 'share of Graduate and Above',
       'average_schooling','Weight', 'TotalExpense']]

In [175]:
HH_Data_v2.to_excel('HH_Data_new_v2120824.xlsx', index= False)

In [177]:
HH_Data_v2.TotalExpense

0         30903.434442
1         24208.706458
2         31282.904110
3         25315.976517
4         17065.553816
              ...     
261741    25161.694716
261742    32385.309198
261743    28774.367906
261744    25981.966732
261745    14288.782779
Name: TotalExpense, Length: 261746, dtype: float64

In [871]:
#Exploratory Data Analysis

In [179]:
import sweetviz as sv

my_report = sv.analyze(HH_Data_v2)
my_report.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [767]:
import dtale

d = dtale.show(HH_Data_v2)
d

In [771]:
from ydata_profiling import ProfileReport

profile = ProfileReport(HH_Data_v2, title="Profiling Report")

2024-10-17 13:43:25,318 - INFO     - Pandas backend loaded 2.2.2
2024-10-17 13:43:25,341 - INFO     - Numpy backend loaded 1.26.4
2024-10-17 13:43:25,350 - INFO     - Pyspark backend NOT loaded
2024-10-17 13:43:25,350 - INFO     - Python backend loaded


In [181]:
import pandas as pd
import numpy as np

# Function to calculate statistics for numeric variables
def numeric_statistics(df, column):
    stats_dict = {}
    stats_dict['Variable Type'] = 'Numeric'
    stats_dict['Count'] = df[column].count()  # Number of non-missing values
    stats_dict['Missing Values'] = df[column].isnull().sum()
    stats_dict['Mean'] = df[column].mean()
    stats_dict['Median'] = df[column].median()
    stats_dict['Mode'] = df[column].mode()[0] if not df[column].mode().empty else None
    stats_dict['Min'] = df[column].min()
    stats_dict['Max'] = df[column].max()
    
    # Deciles
    deciles = np.percentile(df[column], np.arange(0, 101, 10))
    for i, decile in enumerate(deciles):
        stats_dict[f'Decile_{i+1}'] = decile  # Label deciles from 1 to 10
    
    # Percentiles
    percentiles = np.percentile(df[column], [5, 25, 50, 75,95])
    stats_dict['Percentile_5'] = percentiles[0]
    stats_dict['Percentile_25'] = percentiles[1]
    stats_dict['Percentile_50'] = percentiles[2]
    stats_dict['Percentile_75'] = percentiles[3]
    stats_dict['Percentile_95'] = percentiles[4]
    
    return stats_dict

# Function to calculate statistics for categorical variables
def categorical_statistics(df, column):
    stats_dict = {}
    stats_dict['Variable Type'] = 'Categorical'
    stats_dict['Count'] = df[column].count()  # Number of non-missing values
    stats_dict['Missing Values'] = df[column].isnull().sum()
    stats_dict['Mode'] = df[column].mode()[0] if not df[column].mode().empty else None
    return stats_dict

# Function to perform EDA based on dtypes
def eda_statistics(df):
    eda_results = {}

    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            eda_results[column] = numeric_statistics(df, column)
        elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):
            eda_results[column] = categorical_statistics(df, column)
        else:
            eda_results[column] = "Non-numeric and non-categorical type"

    return pd.DataFrame(eda_results).T

# Generate the EDA statistics
eda_results = eda_statistics(HH_Data_v2)


C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1693380996.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1693380996.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):
C:\Users\Hp\AppData\Local\Temp\ipykernel_9368\1693380996.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):


In [183]:
eda_results.to_excel(r'C:\Users\Hp\Desktop\HCES2223\Analysis\New Result\eda_results_v1.xlsx')

In [185]:
import pandas as pd

# Function to calculate the correlation matrix for numeric variables
def correlation_matrix(df):
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    # Compute the correlation matrix
    correlation_mat = numeric_df.corr()
    return correlation_mat

# Generate the correlation matrix for numeric variables
corr_matrix = correlation_matrix(HH_Data_v2)
print(corr_matrix)


                                           Age_HH_Head  \
Age_HH_Head                                   1.000000   
Total_year_of_education_completed_HH_head    -0.221837   
Share of HH members in 0-14                  -0.337440   
Share of HH members in 15-24                 -0.198413   
Share of HH members in 25-59                 -0.102691   
Share of HH members in 60 and older           0.589848   
Share of Married Members                      0.189038   
share of Graduate and Above                   0.070380   
average_schooling                             0.004933   
Weight                                       -0.041362   
TotalExpense                                  0.095050   

                                           Total_year_of_education_completed_HH_head  \
Age_HH_Head                                                                -0.221837   
Total_year_of_education_completed_HH_head                                   1.000000   
Share of HH members in 0-14            

In [187]:
corr_matrix=corr_matrix.reset_index()

In [189]:
corr_matrix.to_excel(r'C:\Users\Hp\Desktop\HCES2223\Analysis\New Result\corr_matrix.xlsx', index= False)

In [1]:
import pandas as pd

In [3]:
HH_Data= pd.read_excel(r'C:\Users\Hp\Desktop\HCES2223\Data\HH_Data_new_v2.xlsx')

In [7]:
HH_Data.columns

Index(['HH_ID', 'Sector', 'State', 'NSS-Region', 'District', 'Household Type',
       'Religion_HH_head', 'Social_Group_HH_head', 'HH Size (For FDQ)',
       'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)', 'Gender_HH_Head',
       'Age_HH_Head', 'Marital_Status_HH_head',
       'Total_year_of_education_completed_HH_head',
       'Ind_highest_level_of_eduction_HH_head', 'Share of HH members in 0-14',
       'Share of HH members in 15-24', 'Share of HH members in 25-59',
       'Share of HH members in 60 and older', 'Share of Married Members',
       'share of Graduate and Above', 'average_schooling', 'Weight',
       'TotalExpense', 'WtdTotalExpense'],
      dtype='object')

In [9]:
HH_Data['WtdTotalExpense']=HH_Data['TotalExpense']*HH_Data['Weight']
HH_Data['WtdHH_in_F']=HH_Data['HH Size (For FDQ)']*HH_Data['Weight']

In [1]:
(HH_Data.groupby(['Sector'])['WtdTotalExpense'].sum() / HH_Data.groupby(['Sector'])['WtdHH_in_F'].sum()).reset_index(name='MPCE')

NameError: name 'HH_Data' is not defined

In [ ]:
 # Save the processed data to CSV
output_file = f"C:/Users/Hp/Desktop/HCES2223/CSV/" + file_name +'.parquet'
df.to_parquet(output_file, index=False)